In [1]:
import os, ssl
import requests
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# Covid Cases per City in OC  

In [3]:
url = "https://ochca.maps.arcgis.com/apps/opsdashboard/index.html#/2a169f85c2254dd7b43f95b095208356"
browser.visit(url)

In [4]:
html = browser.html
soup = bs(html,'html.parser')
data = soup.find_all("nav", class_="feature-list")

In [5]:
confirmed_case_data = data[0].find_all("p")
confirmed_case_city = []
for i in confirmed_case_data:
    a = i.text
    b = a.split("-")
    city_name = b[0]
    confirmed_case = b[1].split(" ")[0][1:]
    confirmed_case_city.append([city_name,confirmed_case])

In [6]:
death_case_data = data[1].find_all("p")
death_case_city = []
for i in death_case_data:
    a = i.text
    b = a.split("-")
    city_name = b[0]
    if("Less"in b[1]):
        death_case = b[1][:-12]
    else:
        death_case = b[1].split(" ")[1]
    death_case_city.append([city_name, death_case])  

In [7]:
oc_case_df = pd.DataFrame(confirmed_case_city)
oc_death_df = pd.DataFrame(death_case_city)

In [8]:
oc_case_df.columns = ["City","Confirmed_case"]
oc_death_df.columns = ["City","Death_case"]

In [9]:
orange_df = oc_case_df.merge(oc_death_df,how="inner", on="City")

In [10]:
orange_df["County"]=""
for index,row in orange_df.iterrows():
    orange_df.loc[index,"County"] = "Orange"

In [11]:
orange_county_df = orange_df[["County","City","Confirmed_case","Death_case"]]

In [12]:
orange_county_df.head()

,County,City,Confirmed_case,Death_case
0,Orange,Aliso Viejo,215,1
1,Orange,Anaheim,4792,112
2,Orange,Brea,248,5
3,Orange,Buena Park,765,6
4,Orange,Costa Mesa,865,5


# Covid Cases per City in LA County¶

In [13]:
url = "http://publichealth.lacounty.gov/media/Coronavirus/locations.htm"
browser.visit(url)

In [14]:
html = browser.html
soup = bs(html,'html.parser')
data = soup.find_all("tr")

In [15]:
LA_case = []
for row in data:
    list_td = row.find_all("td")
    temp_list = []
    for item in list_td:
        if(item.text):
            temp_list.append(item.text)
    if(temp_list):
        if "City of " in temp_list[0]:
            city_data = temp_list[0][8:];
            temp_list[0] = city_data;
            LA_case.append(temp_list)

In [16]:
la_case_df = pd.DataFrame(LA_case)
la_case_df.columns= ["City", "Confirmed_case", "Case_Rate", "Death_case", "Death_Rate"]

In [17]:
la_case_df["County"]=""

In [18]:
for index,row in la_case_df.iterrows():
    la_case_df.loc[index,"County"] = "Los Angeles"

In [19]:
la_county_df = la_case_df[["County","City","Confirmed_case","Death_case"]]
la_county_df.head()

,County,City,Confirmed_case,Death_case
0,Los Angeles,Agoura Hills,86,1
1,Los Angeles,Alhambra,674,13
2,Los Angeles,Arcadia,244,12
3,Los Angeles,Artesia,175,3
4,Los Angeles,Avalon,5,0


# Covid Cases per City in San Diego County

In [23]:
import tabula
from tabula import read_pdf

In [24]:
url = "https://www.sandiegocounty.gov/content/dam/sdc/hhsa/programs/phs/Epidemiology/COVID-19%20Daily%20Update_City%20of%20Residence.pdf"

In [25]:
df = read_pdf(url)

'pages' argument isn't specified.Will extract only from page 1 by default.


In [26]:
sandiego_df = df[0].drop([0])

In [27]:
sandiego_df.head()

,Unnamed: 0,"Summary of COVID-19 Cases by City of Residence, San Diego County"
1,Incorporated City,"18,842 87.5% 664.3"
2,Carlsbad,345 1.6% 299.4
3,Chula Vista,"2,824 13.1% 1,040.5"
4,Coronado,96 0.4% 396.7
5,Del Mar,24 0.1% ***


In [28]:
for index,row in sandiego_df.iterrows():
    split_data = row[1].split()
    length = len(split_data)
    count = split_data[0]
    if(length==3):
        percentage_total = split_data[1]
        rate_per_100000 = split_data[2]
    elif (length==2):
        percentage_total = split_data[1]
        rate_per_100000 = ""
    elif(length==1):
        percentage_total = ""
        rate_per_100000 = ""
    sandiego_df.loc[index,"count"] = count
    sandiego_df.loc[index,"percentage_total"] = percentage_total
    sandiego_df.loc[index,"rate_per_100000"] = rate_per_100000

In [29]:
sandiego_df.columns = ["City", "Summary", "Confirmed_case", "Total_Rate", "Rate_per_10000"]

In [30]:
sandiego_df["County"]=""
sandiego_df["Death_case"]=""

In [31]:
for index,row in sandiego_df.iterrows():
    sandiego_df.loc[index,"County"] = "San Diego"

In [32]:
sandiego_county_df = sandiego_df[["County","City","Confirmed_case","Death_case"]]
sandiego_county_df.head()

,County,City,Confirmed_case,Death_case
1,San Diego,Incorporated City,"18,842",
2,San Diego,Carlsbad,345,
3,San Diego,Chula Vista,"2,824",
4,San Diego,Coronado,96,
5,San Diego,Del Mar,24,


# Covid Cases per City in Santa Barbara County

In [33]:
url = "https://publichealthsbc.org/status-reports/"
browser.visit(url)

In [34]:
html = browser.html
soup = bs(html,'html.parser')
data = soup.find_all("table")

In [35]:
a = data[0].find_all("tr")

In [36]:
sb_list = []
for i in a:
    j = i.find_all("td")
    temp_data = []
    for data in j:
        temp_data.append(data.text)
    sb_list.append(temp_data)


In [37]:
sb_df = pd.DataFrame(sb_list)

In [38]:
sb_df["County"]=""
sb_df

,0,1,2,3,4,5,County
0,Geographic Area as of 7/16/2020,DailyCases,Total Confirmed Cases,Recovered by Region,Active Cases by Region,Number of Deaths,
1,SOUTH COUNTY UNINCORPORATED AREA includes comm...,4,85,80,4,0,
2,CITY OF SANTA BARBARA and the unincorporated a...,44,544,491,51,3,
3,CITY OF GOLETA,5,98,89,7,2,
4,COMMUNITY OF ISLA VISTA,1,29,27,2,0,
5,UNINCORPORATED AREA OF THE GOLETA VALLEY AND G...,5,74,66,7,1,
6,SANTA YNEZ VALLEY including the Cities of Solv...,6,49,37,12,0,
7,CITY OF LOMPOC and the communities of Mission ...,19,271,235,32,4,
8,FEDERAL PRISON IN LOMPOC,1,1001,995,3,3,
9,CITY OF SANTA MARIA,112,2036,1792,227,18,


In [39]:
sb_df.columns = ["City", "DailyCases", "Confirmed_case","recovered","active","Death_case","County"]

In [40]:
sb_df.drop([0],inplace=True)

In [41]:
for index,row in sb_df.iterrows():
    sb_df.loc[index,"County"] = "Santa Barbara"

In [42]:
santabarbara_county_df=sb_df[["County","City","Confirmed_case","Death_case"]]
santabarbara_county_df

,County,City,Confirmed_case,Death_case
1,Santa Barbara,SOUTH COUNTY UNINCORPORATED AREA includes comm...,85,0
2,Santa Barbara,CITY OF SANTA BARBARA and the unincorporated a...,544,3
3,Santa Barbara,CITY OF GOLETA,98,2
4,Santa Barbara,COMMUNITY OF ISLA VISTA,29,0
5,Santa Barbara,UNINCORPORATED AREA OF THE GOLETA VALLEY AND G...,74,1
6,Santa Barbara,SANTA YNEZ VALLEY including the Cities of Solv...,49,0
7,Santa Barbara,CITY OF LOMPOC and the communities of Mission ...,271,4
8,Santa Barbara,FEDERAL PRISON IN LOMPOC,1001,3
9,Santa Barbara,CITY OF SANTA MARIA,2036,18
10,Santa Barbara,COMMUNITY OF ORCUTT,138,0


# Covid Cases per City in Riverside County

In [43]:
import tabula
from tabula import read_pdf

In [44]:
url = "https://www.rivcoph.org/Portals/0/Documents/CoronaVirus/July/Dashboard/2/City_Report_071320.pdf?ver=2020-07-13-151405-373&timestamp=1594678461087"
df = read_pdf(url)

'pages' argument isn't specified.Will extract only from page 1 by default.


In [45]:
riverside_df = df[0]
riverside_df["County"]=""

In [46]:
for index,row in riverside_df.iterrows():
    riverside_df.loc[index,"County"] = "Riverside"

In [47]:
riverside_df.columns=["City","Confirmed_case","Death_case","Percentage","County"]

In [48]:
riverside_county_df=riverside_df[["County","City","Confirmed_case","Death_case"]]
riverside_county_df.head()

,County,City,Confirmed_case,Death_case
0,Riverside,County Jails,259,3
1,Riverside,State Prison,"1,156",1
2,Riverside,Incorporated City,"20,889",493
3,Riverside,Banning,284,10
4,Riverside,Beaumont,474,12


# Covid Cases per City in San Bernadino County

In [49]:
import os

In [50]:
url1 = "Resources/COVID-19 PC Surveillance Dashboard_confirmed_files/saved_resource.html"
# url2 = "COVID-19 PC Surveillance Dashboard_death.html"

In [51]:
with open(url1, encoding = 'utf-8') as file:
    html = file.read()   
soup = bs(html,'html.parser')

In [52]:
data = soup.find_all("tbody")

In [53]:
a = data[0].find_all("tr")

In [54]:
list_index = len(a)-1;
san_bernadino_list = [];
b = a[0].find_all("p")
san_bernadino_list.append([b[0].text[2:], b[1].text, b[3].text, b[5].text])
for index in range(list_index):
    sb_value = a[index+1].find_all("p");
    temp_list = [];
    for j in range(len(sb_value)):
        if(j==0):
            temp_list.append(sb_value[j].text[2:])
        else:
            temp_list.append(sb_value[j].text)
    san_bernadino_list.append(temp_list)

In [55]:
san_bernadino_df= pd.DataFrame(san_bernadino_list)

In [56]:
san_bernadino_df.drop([0],inplace=True)

In [57]:
san_bernadino_df.head()

,0,1,2,3
1,Adelanto,259,5,264
2,Angelus Oaks,0,1,1
3,Apple Valley,318,2,320
4,Argus,0,0,0
5,Arrowbear,0,0,0


In [58]:
url2 = "Resources/COVID-19 PC Surveillance Dashboard_death_files/saved_resource.html"
with open(url2, encoding = 'utf-8') as file:
    html = file.read()   
soup = bs(html,'html.parser')

In [59]:
data = soup.find_all("tbody")

In [60]:
a = data[1].find_all("tr")

In [61]:
list_index = len(a)-1;
san_bernadino_list = [];
b = a[0].find_all("p")
san_bernadino_list.append([b[0].text, b[1].text])
for index in range(list_index):
    sb_value = a[index+1].find_all("p");
    temp_list = [];
    for j in range(len(sb_value)):
        if(j==0):
            temp_list.append(sb_value[j].text[0:])
        else:
            temp_list.append(sb_value[j].text)
    san_bernadino_list.append(temp_list)

In [62]:
san_death_df = pd.DataFrame(san_bernadino_list)

In [63]:
san_death_df.drop([0],inplace=True)

In [64]:
san_death_df.head()

,0,1
1,Adelanto,5
2,Apple Valley,2
3,Barstow,2
4,Bloomington,5
5,Blue Jay,1


In [65]:
san_bernadino_df.columns=["Location","Community","Outbreak","Total"]

In [66]:
san_death_df.columns=["Location","Deaths"]

In [67]:
total_san_bernadino_df = san_bernadino_df.merge(san_death_df,how='left', on = 'Location')

In [68]:
total_san_bernadino_df.fillna(0, inplace=True)

In [69]:
total_san_bernadino_df["County"]=""
total_san_bernadino_df.columns=["City","Community","Outbreak","Confirmed_case","Death_case","County"]
total_san_bernadino_df.head()

,City,Community,Outbreak,Confirmed_case,Death_case,County
0,Adelanto,259,5,264,5,
1,Angelus Oaks,0,1,1,0,
2,Apple Valley,318,2,320,2,
3,Argus,0,0,0,0,
4,Arrowbear,0,0,0,0,


In [70]:
for index,row in total_san_bernadino_df.iterrows():
    total_san_bernadino_df.loc[index,"County"] = "San Bernardino"

In [71]:
sanbernadino_county_df=total_san_bernadino_df[["County","City","Confirmed_case","Death_case"]]
sanbernadino_county_df.head()

,County,City,Confirmed_case,Death_case
0,San Bernardino,Adelanto,264,5
1,San Bernardino,Angelus Oaks,1,0
2,San Bernardino,Apple Valley,320,2
3,San Bernardino,Argus,0,0
4,San Bernardino,Arrowbear,0,0


# Total Case for sourthern california except imperial, ventura

In [69]:
total_city_df = pd.concat([orange_county_df,la_county_df,sandiego_county_df,riverside_county_df,sanbernadino_county_df,santabarbara_county_df])

In [70]:
total_city_df.columns=["county","city","totalcountconfirmed","totalcountdeaths"]
total_city_df.head()

,county,city,totalcountconfirmed,totalcountdeaths
0,Orange,Aliso Viejo,206,
1,Orange,Anaheim,4654,
2,Orange,Brea,239,
3,Orange,Buena Park,738,
4,Orange,Costa Mesa,840,


In [71]:
from sqlalchemy import create_engine

In [72]:
rds_connection_string = "postgres:postgres@localhost:5432/covid-vaccine-plz"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [73]:
engine.table_names()

['statewide_table']

In [74]:
total_city_df.to_sql(name='citywide_table',con=engine,if_exists='append',index=False)

In [75]:
pd.read_sql_query('select* from citywide_table',con=engine)

,county,city,totalcountconfirmed,totalcountdeaths
0,Orange,Aliso Viejo,206,
1,Orange,Anaheim,4654,
2,Orange,Brea,239,
3,Orange,Buena Park,738,
4,Orange,Costa Mesa,840,
...,...,...,...,...
292,Santa Barbara,COMMUNITY OF ORCUTT,134,0
293,Santa Barbara,"UNINCORPORATED AREAS of Sisquoc, Casmalia, Gar...",172,0
294,Santa Barbara,Out of County,—,1
295,Santa Barbara,Pending,116,—


In [76]:
total_city_df.to_csv("./CSV/citywide.csv", index=False)